In [1]:
import json
import os

from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import AzureOpenAI

Load environment variables (needed for Azure OpenAI)

In [2]:
load_dotenv()

True

Setup Azure OpenAI client

In [3]:
client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_URL"],
    api_key=os.environ["AZURE_OPENAI_KEY"],
    api_version="2024-02-01",
)

Load in an example publication to summarize.

In [4]:
with open("resources/publication.json", "r") as f:
    publication = json.load(f)
publication

{'vat': '799497754',
 'pubid': '23070228',
 'act_description': 'RUBRIEK HERSTRUCTURERING (FUSIE, SPLITSING, OVERDRACHT VERMOGEN, ENZ...)',
 'company_name': 'NORRIQ FINANCIAL SERVICES',
 'company_juridical_form': 'BV',
 'address': 'ZUIDERLAAN 11, BUS 11 8790 WAREGEM',
 'street': 'ZUIDERLAAN 11, BUS 11',
 'zipcode': '8790',
 'city': 'WAREGEM',
 'publication_date': '2023-05-31',
 'publication_number': '0070228',
 'publication_link': 'https://www.ejustice.just.fgov.be/tsv_pdf/2023/05/31/23070228.pdf',
 'text': 'Ondernemingsnr : 0799 497 754\nNaam\n(voluit) : NORRIQ Financial Services\n(verkort) :\nRechtsvorm : Besloten Vennootschap\nVolledig adres v.d. zetel : Zuiderlaan 11, bus 11, 8790 Waregem\nOnderwerp akte : Voorstel tot overdracht van een bedrijfstak van EY Consulting BV aan\nNORRIQ Financial Services BV opgemaakt overeenkomstig artikel 12:103\nvan het Wetboek van Vennootschappen en Verenigingen\nOvereenkomstig de artikelen 12:103 juncto 12:93 van het Wetboek van vennootschappen en v

Define a system message. A system prompt is the first message the chat language model sees. It is trained to adhere to its instructions.

In [5]:
system_message = """
You are a helpful assistant that helps summarize documents.
Start with a short intro which must include:
- company name
- vat number
- juridical form
- address
- publication date, publication number, publication link

Then follow follow with a section where you summarize the publication text. Pay attention to the following topics:
- changes in capital (increase/decrease in shares or change of owner of shares)
- appointment and resignation of legal representatives (persons and organizations that can represent the legal entity towards third parties)
- definition of signing rules (how many legal representatives are required to represent the legal entity towards third parties)
- terminations, mergers & acquisitions and restructurings
- change in general assembly date


Start with listing the topics which are mentioned within the publications (do not include those not mentioned) plus any topics you deem important.
Conclude with listing the topics which were not mentioned within the publications.

Answer in English and use Markdown formatting.
"""

In [6]:
completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    temperature=0,
    messages=[
        {"role": "system", "content": system_message},
        {
            "role": "user",
            "content": f"Summarize me the following publication: {publication}",
        },
    ],
)

In [7]:
display(Markdown(completion.choices[0].message.content))

### Summary of Publication for NORRIQ Financial Services

**Company Information:**
- **Company Name:** NORRIQ Financial Services
- **VAT Number:** 799497754
- **Juridical Form:** BV (Besloten Vennootschap)
- **Address:** Zuiderlaan 11, Bus 11, 8790 Waregem
- **Publication Date:** 2023-05-31
- **Publication Number:** 0070228
- **Publication Link:** [Link to Publication](https://www.ejustice.just.fgov.be/tsv_pdf/2023/05/31/23070228.pdf)

### Summary of Publication Text

**Topics Mentioned:**
- **Terminations, Mergers & Acquisitions and Restructurings:**
  - NORRIQ Financial Services BV is involved in a restructuring process where it is proposed to acquire a business division from EY Consulting BV. This transaction is structured as a transfer of assets and liabilities that can operate autonomously, technically, and organizationally.
  - The transfer is to be executed under the regulations outlined in articles 12:93 to 12:100 of the Companies and Associations Code, making the transfer effective by law, transferring all connected assets and liabilities to NORRIQ Financial Services BV.
  - The personnel employed exclusively in the transferred business division will be employed by NORRIQ Financial Services BV post-transfer.

**Important Additional Information:**
- **Legal and Regulatory Compliance:**
  - The transfer proposal has been drafted and approved in accordance with the relevant articles of the Companies and Associations Code.
  - Formalities for the transfer, including the deposition of the proposal at the competent enterprise court, have been delegated to Ad-Ministerie BV, represented by Adriaan de Leeuw.

**Topics Not Mentioned:**
- Changes in capital (increase/decrease in shares or change of owner of shares)
- Appointment and resignation of legal representatives
- Definition of signing rules
- Change in general assembly date

This summary provides an overview of the key restructuring activity involving NORRIQ Financial Services as detailed in the publication, focusing on the acquisition of a business division from EY Consulting BV.